In [6]:
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import random
import itertools

# Task 1

In [8]:
def read_edges(file_path):
    with open(file_path, 'r') as file:
        edges = [tuple(map(float, line.strip().split())) for line in file]
    return edges

def visualize_network(edges):
    G = nx.Graph()
    for edge in edges:
        if len(edge) == 3:
            u, v, weight = edge
            G.add_edge(int(u), int(v), weight=weight)
        else:
            u, v = edge
            G.add_edge(int(u), int(v))
    
    pos = nx.spring_layout(G, seed=42)
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')
    
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
    
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=[str(node) for node in G.nodes()],
        textposition="top center",
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))
    
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Network Visualization',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[dict(
                            text="Network visualization using Plotly",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 )],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False)))
    
    fig.show()

file_path = 'ENZYMES_g1.edges'
edges = read_edges(file_path)
visualize_network(edges)

In [9]:
def read_facebook_network(file_path):
    G = nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)
    return G

def degree_distribution(G):
    degrees = [G.degree(n) for n in G.nodes()]
    degree_count = np.bincount(degrees)
    pk = degree_count / len(G.nodes())
    avg_degree = np.mean(degrees)
    return pk, avg_degree

def clustering_coefficients(G):
    clustering = nx.clustering(G)
    clustering_values = list(clustering.values())
    avg_clustering = np.mean(clustering_values)
    return clustering_values, avg_clustering

def shortest_paths(G):
    lengths = dict(nx.all_pairs_shortest_path_length(G))
    all_lengths = []
    for node in lengths:
        all_lengths.extend(lengths[node].values())
    diameter = max(all_lengths)
    avg_path_length = np.mean(all_lengths)
    return all_lengths, diameter, avg_path_length

def plot_distributions(pk, clustering_values, all_lengths, title_prefix):
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Degree Distribution", "Clustering Coefficient Distribution", "Shortest Path Length Distribution"))

    degree_trace = go.Bar(x=list(range(len(pk))), y=pk)
    fig.add_trace(degree_trace, row=1, col=1)
    fig.update_xaxes(title_text="Degree k", row=1, col=1)
    fig.update_yaxes(title_text="P(k)", row=1, col=1)

    clustering_trace = go.Histogram(x=clustering_values, nbinsx=30)
    fig.add_trace(clustering_trace, row=1, col=2)
    fig.update_xaxes(title_text="Clustering Coefficient", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)

    path_trace = go.Histogram(x=all_lengths, nbinsx=30)
    fig.add_trace(path_trace, row=1, col=3)
    fig.update_xaxes(title_text="Path Length", row=1, col=3)
    fig.update_yaxes(title_text="Frequency", row=1, col=3)

    fig.update_layout(title_text=f"{title_prefix} - Distributions", showlegend=False)
    fig.show()


# Task 2

In [ ]:
file_path = 'facebook_combined.txt'
G = read_facebook_network(file_path)

pk, avg_degree = degree_distribution(G)
clustering_values, avg_clustering = clustering_coefficients(G)
all_lengths, diameter, avg_path_length = shortest_paths(G)

print(f"Facebook Network - Average Degree <k>: {avg_degree}")
print(f"Facebook Network - Average Clustering Coefficient: {avg_clustering}")
print(f"Facebook Network - Diameter: {diameter}")
print(f"Facebook Network - Average Path Length: {avg_path_length}")

plot_distributions(pk, clustering_values, all_lengths, "Facebook Network")

In [10]:
def erdos_renyi_gnl(N, L):
    G = nx.Graph()
    G.add_nodes_from(range(N))
    possible_edges = list(itertools.combinations(range(N), 2))
    random_edges = random.sample(possible_edges, L)
    G.add_edges_from(random_edges)
    return G

def erdos_renyi_gnp(N, p):
    G = nx.gnp_random_graph(N, p)
    return G

def watts_strogatz(N, k, beta):
    G = nx.watts_strogatz_graph(N, k, beta)
    return G

# Task 3

In [11]:

N = 100  # Number of nodes
L = 200  # Number of edges for Erdős-Rényi model G(N, L)
p = 0.05  # Probability for Erdős-Rényi-Gilbert model G(N, p)
k = 4  # Each node is connected to k nearest neighbors in ring topology for Watts-Strogatz model
beta = 0.1  # Rewiring probability for Watts-Strogatz model

G_gnl = erdos_renyi_gnl(N, L)
G_gnp = erdos_renyi_gnp(N, p)
G_ws = watts_strogatz(N, k, beta)

# Erdős-Rényi model
pk_gnl, avg_degree_gnl = degree_distribution(G_gnl)
clustering_values_gnl, avg_clustering_gnl = clustering_coefficients(G_gnl)
all_lengths_gnl, diameter_gnl, avg_path_length_gnl = shortest_paths(G_gnl)
print(f"Erdős-Rényi G(N, L) - Average Degree <k>: {avg_degree_gnl}")
print(f"Erdős-Rényi G(N, L) - Average Clustering Coefficient: {avg_clustering_gnl}")
print(f"Erdős-Rényi G(N, L) - Diameter: {diameter_gnl}")
print(f"Erdős-Rényi G(N, L) - Average Path Length: {avg_path_length_gnl}")
plot_distributions(pk_gnl, clustering_values_gnl, all_lengths_gnl, "Erdős-Rényi G(N, L)")

# Erdős-Rényi-Gilbert model
pk_gnp, avg_degree_gnp = degree_distribution(G_gnp)
clustering_values_gnp, avg_clustering_gnp = clustering_coefficients(G_gnp)
all_lengths_gnp, diameter_gnp, avg_path_length_gnp = shortest_paths(G_gnp)
print(f"Erdős-Rényi-Gilbert G(N, p) - Average Degree <k>: {avg_degree_gnp}")
print(f"Erdős-Rényi-Gilbert G(N, p) - Average Clustering Coefficient: {avg_clustering_gnp}")
print(f"Erdős-Rényi-Gilbert G(N, p) - Diameter: {diameter_gnp}")
print(f"Erdős-Rényi-Gilbert G(N, p) - Average Path Length: {avg_path_length_gnp}")
plot_distributions(pk_gnp, clustering_values_gnp, all_lengths_gnp, "Erdős-Rényi-Gilbert G(N, p)")

# Watts-Strogatz model
pk_ws, avg_degree_ws = degree_distribution(G_ws)
clustering_values_ws, avg_clustering_ws = clustering_coefficients(G_ws)
all_lengths_ws, diameter_ws, avg_path_length_ws = shortest_paths(G_ws)
print(f"Watts-Strogatz WS(N, k, beta) - Average Degree <k>: {avg_degree_ws}")
print(f"Watts-Strogatz WS(N, k, beta) - Average Clustering Coefficient: {avg_clustering_ws}")
print(f"Watts-Strogatz WS(N, k, beta) - Diameter: {diameter_ws}")
print(f"Watts-Strogatz WS(N, k, beta) - Average Path Length: {avg_path_length_ws}")
plot_distributions(pk_ws, clustering_values_ws, all_lengths_ws, "Watts-Strogatz WS(N, k, beta)")

Erdős-Rényi G(N, L) - Average Degree <k>: 4.0
Erdős-Rényi G(N, L) - Average Clustering Coefficient: 0.024341269841269848
Erdős-Rényi G(N, L) - Diameter: 7
Erdős-Rényi G(N, L) - Average Path Length: 3.3021674458138546


Erdős-Rényi-Gilbert G(N, p) - Average Degree <k>: 5.16
Erdős-Rényi-Gilbert G(N, p) - Average Clustering Coefficient: 0.0723095238095238
Erdős-Rényi-Gilbert G(N, p) - Diameter: 7
Erdős-Rényi-Gilbert G(N, p) - Average Path Length: 2.9634


Watts-Strogatz WS(N, k, beta) - Average Degree <k>: 4.0
Watts-Strogatz WS(N, k, beta) - Average Clustering Coefficient: 0.3453333333333333
Watts-Strogatz WS(N, k, beta) - Diameter: 10
Watts-Strogatz WS(N, k, beta) - Average Path Length: 4.8966
